In [ ]:
import pandas as pd
import numpy as np
import itertools
import random
import math
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from functools import partial
from statistics import mean, stdev
from sklearn.naive_bayes import GaussianNB
from math import dist
from itertools import product
from collections import Counter, OrderedDict
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from statistics import mean, stdev

from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.knn import KNN

from sklearn.naive_bayes import GaussianNB
from sklearn_extra.cluster import KMedoids
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors as NN
from sklearn.ensemble import RandomForestClassifier

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None 


In [ ]:
import lightgbm as ltb
from scipy.stats import chi2_contingency

In [ ]:
import time
import statistics


In [ ]:
from scipy.spatial import distance

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler


In [ ]:
from sdv.evaluation import evaluate as eva

from pyod.models.lof import LOF
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.knn import KNN

from math import dist


In [ ]:
def ranker(df, attributes, class_name):
    X = df[attributes].values
    y = df[class_name].to_list()
    clf = GaussianNB()

    clf.fit(X, y)
    X_proba = [None] * len(X)
    probability = []
    for k in range(len(X)):
        X_proba[k] = np.append(X[k], (y[k], clf.predict_proba(X)[k][1]))

    X_proba = np.array(X_proba, dtype=float).tolist()
    
    X_proba_list = []
    
    for arr in (X_proba):
        X_proba_list.append(arr[:len(attributes)+2])
        
    X_proba = pd.DataFrame.from_records(X_proba_list)
    X_proba.columns = attributes + [class_name] + ['proba']
    
    return X_proba

In [ ]:
def get_index (attribute, attributes):
    for i in range(len(attributes)):
        if attributes[i] == attribute:
            return i

In [ ]:
def get_closest_value (val, value_list):
    
    return value_list[np.argmin(np.abs(np.array(value_list)-val))]

In [ ]:
def get_causal_regressor (data, index_X, index_y):
    
    X = data.iloc[:, index_X].values
    y = data.iloc[:, [index_y]].values

    regr = ltb.LGBMRegressor().fit(X, y)

    return regr

In [ ]:
def get_causal_classifier (data, index_X, index_y):

    X = data.iloc[:, index_X].values
    y = data.iloc[:, [index_y]].values

    classifier = ltb.LGBMClassifier().fit(X, y)    

    return classifier

In [ ]:
def get_discrimination (df, sensitive_attributes, class_name):

# ASSUMPTION: Each value of the attribute is discriminated
# For each value, we therefore apply the Preferential Sampling formulas to compute the discrimination 
# If discrimination > 0, the assumption holds true
# Otherwhise, it doesn't. This means the value is actually *privileged*
# A dictionary of sensitive attributes and values is created as such
#
# Please note the sum of records to add / remove for each priviliged value
# should be equal to the sum of records to add / removed for each discriminated value
#
# A rounding error is possible

    
    sensitive_dict = {}
    
    tot_disc = 0
    tot_pos = 0
    
    #df = X_proba
    
    for attr in sensitive_attributes:
        print ()
        print ("Analizing", attr, "...")
        sensitive_dict[attr] = {}
        sensitive_dict[attr]['D'] = {}
        sensitive_dict[attr]['P'] = {}
        sensitive_dict[attr]['D']['values_list'] = []
        sensitive_dict[attr]['P']['values_list'] = []
        values = df[attr].unique()
        for val in values:
            PP = df[(df[attr] != val) & (df[class_name] == 1)].values.tolist()
            PN = df[(df[attr] != val) & (df[class_name] == 0)].values.tolist()
            DP = df[(df[attr] == val) & (df[class_name] == 1)].values.tolist()
            DN = df[(df[attr] == val) & (df[class_name] == 0)].values.tolist()

            disc = len(DN) + len(DP)
            priv = len(PN) + len(PP)
            pos = len(PP) + len(DP)
            neg = len(PN) + len(DN)
            
            DP_exp = round(disc * pos / len(df))
            PP_exp = round(priv * pos / len(df))
            DN_exp = round(disc * neg / len(df))
            PN_exp = round(priv * neg / len(df))
            
            discrimination = len(PP) / (len(PP) + len(PN)) - len(DP) / (len(DP) + len(DN))
       
            if discrimination >= 0:
                status = 'D'
                sensitive_dict[attr][status][val] = {}
                print("")
                print(val, "is discriminated:", discrimination)
                
                sensitive_dict[attr][status][val]['P'] = sorted(DP, key=lambda x:x[len(DP[0])-1])
                sensitive_dict[attr][status][val]['P_exp'] = DP_exp
                sensitive_dict[attr][status][val]['P_curr'] = 0
                
                for i in range(len(sensitive_dict[attr][status][val]['P'])):
                    del sensitive_dict[attr][status][val]['P'][i][-1]
                                
                sensitive_dict[attr][status][val]['N'] = sorted(DN, key=lambda x:x[len(DN[0])-1], reverse = True)
                sensitive_dict[attr][status][val]['N_exp'] = DN_exp
                sensitive_dict[attr][status][val]['N_curr'] = 0

                for i in range(len(sensitive_dict[attr][status][val]['N'])):
                    del sensitive_dict[attr][status][val]['N'][i][-1]
                    
                print("- DP:", len(sensitive_dict[attr][status][val]['P']), '· Expected:', DP_exp, 
                      '· To be added:', abs(len(DP) - DP_exp))
                print("- DN:", len(sensitive_dict[attr][status][val]['N']), '· Expected:', DN_exp, 
                      '· To be removed:', abs(len(DN) - DN_exp))
                
                tot_disc = tot_disc + abs(len(DP) - DP_exp)
                
            else:
                status = 'P'
                sensitive_dict[attr][status][val] = {}
                print("")
                print(val, "is privileged:", discrimination)   
                
                sensitive_dict[attr][status][val]['P'] = sorted(DP, key=lambda x:x[len(DP[0])-1])
                sensitive_dict[attr][status][val]['P_exp'] = DP_exp
                sensitive_dict[attr][status][val]['P_curr'] = 0

                for i in range(len(sensitive_dict[attr][status][val]['P'])):
                    del sensitive_dict[attr][status][val]['P'][i][-1]
                    
                sensitive_dict[attr][status][val]['N'] = sorted(DN, key=lambda x:x[len(DN[0])-1], reverse = True)
                sensitive_dict[attr][status][val]['N_exp'] = DN_exp
                sensitive_dict[attr][status][val]['N_curr'] = 0

                for i in range(len(sensitive_dict[attr][status][val]['N'])):
                    del sensitive_dict[attr][status][val]['N'][i][-1]
                
                print("- PP:", len(sensitive_dict[attr][status][val]['P']), '· Expected:', DP_exp, 
                      '· To be removed:', abs(len(DP) - DP_exp))
                print("- PN:", len(sensitive_dict[attr][status][val]['N']), '· Expected:', DN_exp, 
                      '· To be added:', abs(len(DN) - DN_exp))
                
                tot_pos = tot_pos + abs(len(DP) - DP_exp)
            
            sensitive_dict[attr][status]['values_list'].append(val)
        
    round_error = abs(tot_disc - tot_pos)
    
    if round_error > 0:
        print ("")
        print ("Due to a rounding error, the final dataset might be slightly smaller")
                                   
    return sensitive_dict            

In [ ]:
def random_individual(values, const, 
                      values_in_dataset_indexes, discrete_indexes, regular_indexes, 
                      causal_reg, causal_class, ds):
    
    df = pd.DataFrame(values)
    
    ind = [None] * (len(df.columns)-1)
        
    for i in regular_indexes:
        val = df.iloc[:, i].to_list()
        
        if i in values_in_dataset_indexes: #if the feat can only assume values already in the dataset
            if ds == 'Random': #if values are picked randomly
                ind[i] = random.choice(list(set(val)))
            elif ds == 'Fixed': #if values are picked w.r.t. their frequency
                ind[i] = random.choice(val)
        elif i in discrete_indexes: #if the feat can only assume a random value in a int range
            ind[i] = random.randint(min(val), max(val))            
        else: #if the feat can assume a float value in a range
            ind[i] = random.uniform(min(val), max(val))
            
    for tup in const:
        ind[tup[0]] = tup[1]
    
    for e in causal_reg + causal_class:
        
        X_indexes = e[0]
        y = e[1]
        pred = e[2]
        
        X_val = []
        for index in X_indexes:
            X_val.append(ind[index])
             
        predicted = pred.predict([X_val])
        
        if y in values_in_dataset_indexes:
            value_list = df.iloc[:, y].to_list()
            ind[y] = get_closest_value(predicted[0], value_list)        
        elif y in discrete_indexes:
            ind[y] = int(predicted[0])        
        else: 
            ind[y] = predicted[0]
    
    return ind

In [ ]:
def evaluate(individual, forest, medoid, mode):
 
    individual = individual[0]
        
    #print (medoid)
    #print ("IND:", individual)
    if mode == "Outlier":
        score = float(forest.predict_proba(np.array([individual]))[0][1])
    elif mode == "Distance":
        if None in medoid:
            score = float(forest.predict_proba(np.array([individual]))[0][1])
        else:
            score = distance.cosine(individual, medoid) / 2
    elif mode == 'Hybrid':
        score = float(forest.predict_proba(np.array([individual]))[0][1])
        if None not in medoid:
            score = (score + distance.cosine(individual, medoid) / 2) / 2
    
    return score,

In [ ]:
def mate(ind1, ind2, values, 
         values_in_dataset_indexes, discrete_indexes, regular_indexes, 
         causal_reg, causal_class):
    
    #custom crossover
    
    indpb = 0.34
    
    for i in regular_indexes:
        if random.random() < indpb:
            ind1[i], ind2[i] = ind2[i], ind1[i]
            for e in causal_reg + causal_class:
                if i in e[0]:
                    df = pd.DataFrame(values)
                    
                    X_indexes = e[0]
                    y = e[1]
                    pred = e[2]

                    X_val1 = []
                    X_val2 = []
                    for index in X_indexes:
                        X_val1.append(ind1[index])
                        X_val2.append(ind2[index])

                    #ind1
                    predicted1 = pred.predict([X_val1])

                    if y in values_in_dataset_indexes:
                        value_list = df.iloc[:, y].to_list()
                        ind1[y] = get_closest_value(predicted1[0], value_list)        
                    elif y in discrete_indexes:
                        ind1[y] = int(predicted1[0])        
                    else: 
                        ind1[y] = predicted1[0]
                        
                    #ind2
                    predicted2 = pred.predict([X_val2])

                    if y in values_in_dataset_indexes:
                        value_list = df.iloc[:, y].to_list()
                        ind2[y] = get_closest_value(predicted2[0], value_list)        
                    elif y in discrete_indexes:
                        ind2[y] = int(predicted2[0])        
                    else: 
                        ind2[y] = predicted2[0]
                        
                    
                    #print ("CHILD1:", ind1)
                    #print ("X:", X_indexes, X_val1, "y:", y, predicted1, predicted1[0][0], ind1[y])
                    #print ("CHILD2:", ind2)
                    #print ("X:", X_indexes, X_val2, "y:", y, predicted2, predicted2[0][0], ind2[y])                    
                
    return ind1, ind2

In [ ]:
def mutate(individual, values, 
           values_in_dataset_indexes, discrete_indexes, regular_indexes, 
           causal_reg, causal_class, ds):
    
#custom mutation

    df = pd.DataFrame(values)
    
    i = random.choice(regular_indexes) #we select a random feature to mutate
    

    if i in values_in_dataset_indexes: 
        val = df.iloc[:, i].to_list()
        
        if ds == "Random":
            individual[i] = random.choice(list(set(val)))
        elif ds == "Fixed":
            individual[i] = random.choice(val)
    elif i in discrete_indexes: #if the feat can only assume a random value in a int range
        val = [x for x in df.iloc[:, i].to_list() if x != individual[i]]
        individual[i] = random.randint(min(val), max(val))                        
    else: #if the feat can assume a float value in a range
        val = [x for x in df.iloc[:, i].to_list() if x != individual[i]]
        individual[i] = random.uniform(min(val), max(val))

    
    for e in causal_reg + causal_class: 
        if i in e[0]:
            #print ("Independent value mutated...")

            X_indexes = e[0]
            y = e[1]
            pred = e[2]

            X_val = []
            for index in X_indexes:
                X_val.append(individual[index])

            predicted = pred.predict([X_val])

            if y in values_in_dataset_indexes:
                value_list = df.iloc[:, y].to_list()
                individual[y] = get_closest_value(predicted[0], value_list)        
            elif y in discrete_indexes:
                individual[y] = int(predicted[0])        
            else: 
                individual[y] = predicted[0]
            
            #print ("X:", X_indexes, X_val, "y:", y, predicted, predicted[0][0], individual[y])    

    #print ("NEW MUTATED:", individual)

    return individual,

In [ ]:
def GA(values, const, n_HOF, forest, medoid, 
       values_in_dataset_indexes, discrete_indexes, regular_indexes, 
       causal_reg, causal_class, mode, ds):
    
    print ("GA started,", n_HOF, "individual(s) will be generated")
        
    NUM_GENERATIONS = 50
    POPULATION_SIZE = 150
    
    CXPB, MUTPB = 0.5, 0.15
    
    creator.create('Fitness', base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.Fitness)

    toolbox = base.Toolbox()

    toolbox.register("random_individual", random_individual, values, const, values_in_dataset_indexes, 
                     discrete_indexes, regular_indexes, causal_reg, causal_class, ds=ds)

    toolbox.register("individual", tools.initRepeat, creator.Individual, 
                     toolbox.random_individual, n=1)

    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("evaluate", evaluate, forest=forest, medoid=medoid, mode=mode)
        
    toolbox.register("mate", mate, values=values, values_in_dataset_indexes=values_in_dataset_indexes, 
                     discrete_indexes=discrete_indexes, regular_indexes=regular_indexes, 
                     causal_reg=causal_reg, causal_class=causal_class)
    
    toolbox.register("mutate", mutate, values=values, values_in_dataset_indexes=values_in_dataset_indexes, 
                     discrete_indexes=discrete_indexes, regular_indexes=regular_indexes, 
                     causal_reg=causal_reg, causal_class=causal_class, ds=ds)    
    
    toolbox.register("select", tools.selNSGA2)



    pop = toolbox.population(n=POPULATION_SIZE)

    hof = tools.HallOfFame(n_HOF)
    #hof = tools.ParetoFront()
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)   
    #stats.register('max', np.max, axis = 0)
    stats.register('min', np.min) #, axis = 0)
    stats.register('avg', np.mean) #, axis = 0)
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + stats.fields
    
    invalid_individuals = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_individuals)
    for ind, fit in zip(invalid_individuals, fitnesses):
        ind.fitness.values = fit
        
    hof.update(pop)
    hof_size = len(hof.items)

    record = stats.compile(pop)
    logbook.record(gen=0, best="-", nevals=len(invalid_individuals), **record)
    print(logbook.stream)

    
    for gen in range(1, NUM_GENERATIONS + 1):
        
                # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values
            
                
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
    
        # Update the hall of fame with the generated individuals
        hof.update(offspring)
        
        # Replace the current population by the offspring
        pop[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(pop) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        print(logbook.stream)
        
        
    hof.update(pop)
    
    counter = 0
    for e in hof.items:
        counter = counter + 1
        print("")
        print("#"+str(counter))
        print("Constraints:", const)
        print('Individual:', e)
        print('Fitness:', e.fitness)


    plt.figure(1)

    # plot genetic flow statistics:
    minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
    plt.figure(2)
    sns.set_style("whitegrid")
    plt.plot(minFitnessValues, color='blue')
    plt.plot(meanFitnessValues, color='green')
    plt.xlabel('Generation')
    plt.ylabel('Fitness Value')
    plt.title('Avg and Min Fitness')
    # show both plots:
    plt.show()
    
    
    return hof.items

In [ ]:
def main (df, sensitive_attributes, class_name, causal_reg, causal_class,
                           discrete_attributes, values_in_dataset_attributes, mode, ds):
    print ("FairGen is running...")
    print ("")
    print ("Target variable:", class_name)
    print ("Sensitive attributes:", sensitive_attributes)
    print ("")
    print ("Discrete attributes:", discrete_attributes)
    print ("Attributes with only dataset values:", values_in_dataset_attributes)
    print ("")
    print ("Regressor:", causal_reg)
    print ("Classifier:", causal_class)
    
    genetic_data = []
        
    values_in_dataset_indexes = []
    
    discrete_indexes = []
    
    regular_indexes = []

    causal_reg_attributes = []
    
    causal_class_attributes = []
            
    attributes = [col for col in df.columns if col != class_name]
    
    
    val_comb = []
    for att in sensitive_attributes:
        val_comb.append(list(df[att].unique()))
    
    df_combinations = list(product(*val_comb))
    
    print ("Ranking the data...")
    
    X_proba = ranker(df, attributes, class_name)
    
    start_time = time.time()
    
    edges = []
    
    if len(causal_reg) > 0:
         
        for e in causal_reg:
            
            X_index = []
            for feat in e[0]:
                X_index.append(get_index(feat, attributes))
                edges.append((feat, e[1]))
            y_index = get_index(e[1], attributes)
            causal_reg_attributes.append(e[1])
            
            regressor = get_causal_regressor (X_proba, X_index, y_index)

            e[0] = X_index
            e[1] = y_index
            e.append(regressor)
            
            # Each e in causal has:
            # e[0] = name of ind variable
            # e[1] = name of dep variable
            # we turned e[0] and e[1] into the variable indexes and we added
            # e[2] = predictor
    
    if len(causal_class) > 0:

        for e in causal_class:

            X_index = []
            for feat in e[0]:
                X_index.append(get_index(feat, attributes))
                edges.append((feat, e[1]))
            y_index = get_index(e[1], attributes)
            causal_class_attributes.append(e[1])

            classifier = get_causal_classifier (X_proba, X_index, y_index)

            e[0] = X_index
            e[1] = y_index
            e.append(classifier)       
    
    nodes = df.columns
   
    dag = nx.DiGraph(edges)

    dag.add_nodes_from(nodes)

    nx.draw_networkx(dag, pos = nx.circular_layout(dag), font_size=10, node_size=350, node_color='#abdbe3')

    plt.title('Assumed Ground Truth', fontsize=13)

    plt.show()
            
    values = X_proba.copy()
    
    target = None
     
    #PART 1 - Discrimination Test
    
    print ("Creating the sensitive dictionary...")
    
    sensitive_dict = get_discrimination (X_proba, sensitive_attributes, class_name)
    
    X_proba = X_proba.iloc[:, :-1] #removing the last column (with the ranker proba)
    
    og_df = X_proba.copy() 

    
    #Training outlier detection methods for fitness
    
    forest=IForest()

    forest.fit(X_proba)
        
    X_proba = X_proba.values
            
    for att in discrete_attributes:
        discrete_indexes.append(get_index(att, attributes)) 

    for att in values_in_dataset_attributes:
        values_in_dataset_indexes.append(get_index(att, attributes))
        
    for att in attributes:
        if att not in sensitive_attributes + causal_reg_attributes + causal_class_attributes:
            regular_indexes.append((get_index(att, attributes)))
            
    X_proba = X_proba.tolist() #list of every record in the dataset

    print("")
    print("Removing records...")
    print("")
            
    
    #### PART2 - Record Removal [of DN / PP]
    
    # We're working with X_proba (list of records) and sensitive_dictionary (various informations about records)

    
    record_informations = []
    # for every record removed, we append a "record information"
    # a "record information" is a tuple of length == len(sensitive_attributes)
    # each value in the tuple is the value the respective attribute had in the removed record
    
    # additionally, we'll make sure to remove the record in every list of records we have

    for att in sensitive_attributes:
        #### Removing DN
        for val in sensitive_dict[att]['D']['values_list']:
            if len(sensitive_dict[att]['D'][val]['N']) > sensitive_dict[att]['D'][val]['N_exp']:
                to_remove = len(sensitive_dict[att]['D'][val]['N']) - sensitive_dict[att]['D'][val]['N_exp']
                for record in sensitive_dict[att]['D'][val]['N'][:to_remove]:
                    record_info = []
                    for att2 in sensitive_attributes:
                        index = get_index(att2, attributes)
                        value = record[index]
                        record_info.append(value)
                        if att2 != att:
                            for val2 in sensitive_dict[att2]['D']['values_list']:
                                if record in sensitive_dict[att2]['D'][val2]['P']:
                                    sensitive_dict[att2]['D'][val2]['P'].remove(record)
                                if record in sensitive_dict[att2]['D'][val2]['N']:
                                    sensitive_dict[att2]['D'][val2]['N'].remove(record)
                            for val2 in sensitive_dict[att2]['P']['values_list']:
                                if record in sensitive_dict[att2]['P'][val2]['P']:
                                    sensitive_dict[att2]['P'][val2]['P'].remove(record)
                                if record in sensitive_dict[att2]['P'][val2]['N']:
                                    sensitive_dict[att2]['P'][val2]['N'].remove(record)                                    
                    record_informations.append(tuple(record_info))
                    X_proba.remove(record)
                sensitive_dict[att]['D'][val]['N'] = sensitive_dict[att]['D'][val]['N'][to_remove:]
        
        ### Removing PP
        for val in sensitive_dict[att]['P']['values_list']:
            if len(sensitive_dict[att]['P'][val]['P']) > sensitive_dict[att]['P'][val]['P_exp']:
                to_remove = len(sensitive_dict[att]['P'][val]['P']) - sensitive_dict[att]['P'][val]['P_exp']
                for record in sensitive_dict[att]['P'][val]['P'][:to_remove]:
                    record_info = []
                    for att2 in sensitive_attributes:
                        index = get_index(att2, attributes)
                        value = record[index]
                        record_info.append(value)
                        if att2 != att:
                            for val2 in sensitive_dict[att2]['D']['values_list']:
                                if record in sensitive_dict[att2]['D'][val2]['P']:
                                    sensitive_dict[att2]['D'][val2]['P'].remove(record)
                                if record in sensitive_dict[att2]['D'][val2]['N']:
                                    sensitive_dict[att2]['D'][val2]['N'].remove(record)
                            for val2 in sensitive_dict[att2]['P']['values_list']:
                                if record in sensitive_dict[att2]['P'][val2]['P']:
                                    sensitive_dict[att2]['P'][val2]['P'].remove(record)
                                if record in sensitive_dict[att2]['P'][val2]['N']:
                                    sensitive_dict[att2]['P'][val2]['N'].remove(record)
                    record_informations.append(tuple(record_info))
                    X_proba.remove(record)
                sensitive_dict[att]['P'][val]['P'] = sensitive_dict[att]['P'][val]['P'][to_remove:]

    if len(record_informations) == 0:
        print ("No records removed! The dataset is already balanced")
        return
        
    print ("Records removed:", len(record_informations)) #each element (list) of record information => 1 removed record
    print ("")
    print ("Current length of dataset: ", len(X_proba))  

    #removed combinations of sensitive attributes:
    common_combs = Counter(tuple(record_informations))
    common_combs = list(OrderedDict(common_combs.most_common()))
    
    # the unique set of "record information", ordered by their frequency
    # i.e. every unique combinations of sensitive attributes removed

    
    for comb in df_combinations:
        if comb not in common_combs:
            common_combs.append(comb)
            
    # additional unique combinations of sensitive attributes
    # not among those removed
    # we append them to the very end of the list => lower priority

    
    
    ### PART3 - Combination Test [Getting the constraints]
    
    
    constraints = []
    # a constraint is a binary tuple
    # tuple[0] = index of a feature; tuple[1] = value of the feature
    # we'll make a constraint for the sensitive attribute(s) and the target variable
    
       
    # "N_cur" => current length of the respective subset
    for att in sensitive_attributes:
        for val in sensitive_dict[att]['D']['values_list']:
            sensitive_dict[att]['D'][val]['N_curr'] = len(sensitive_dict[att]['D'][val]['N'])
            sensitive_dict[att]['D'][val]['P_curr'] = len(sensitive_dict[att]['D'][val]['P'])
            DN_curr = sensitive_dict[att]['D'][val]['N_curr']
            DN_exp = sensitive_dict[att]['D'][val]['N_exp']
            DP_curr = sensitive_dict[att]['D'][val]['P_curr']
            DP_exp = sensitive_dict[att]['D'][val]['P_exp']            
        for val in sensitive_dict[att]['P']['values_list']:
            sensitive_dict[att]['P'][val]['N_curr'] = len(sensitive_dict[att]['P'][val]['N'])
            sensitive_dict[att]['P'][val]['P_curr'] = len(sensitive_dict[att]['P'][val]['P'])     
            PN_curr = sensitive_dict[att]['P'][val]['N_curr']
            PN_exp = sensitive_dict[att]['P'][val]['N_exp']
            PP_curr = sensitive_dict[att]['P'][val]['P_curr']
            PP_exp = sensitive_dict[att]['P'][val]['P_exp']            

            
            
    # combinations of sensitive attributes values are ordered according to their frequency in common_combs
    # higher frequency == higher priority
    # for each value in the comb, we check if a new record with *that* value is needed
    # (i.e, if N_curr < N_exp or P_curr < P_exp)
    # if every value in the combination do pass the check (either for a negative record or a positive record)
    # we'll make a constraint with those values (and the target variable)
    # we repeat those steps as long as neither a negative or a positive record is needed
    # then we'll try the next combination
    # this way, before creating records with a less frequent combination of sens attributes values,
    # we are sure we exhausted the more frequent combinations

    for comb in common_combs:
        while True:
            starting_const_len = len(constraints)
            check = True
            ok_comb_neg = True
            ok_comb_pos = True
            constraint_neg = [(len(attributes), 0)]
            constraint_pos = [(len(attributes), 1)]
            for i in range(len(comb)):
                att = sensitive_attributes[i]
                val = comb[i]
                
                if val in sensitive_dict[att]['D']['values_list']:
                    if sensitive_dict[att]['D'][val]['N_curr'] < sensitive_dict[att]['D'][val]['N_exp']:
                        constraint_neg.append((get_index(att, attributes), val))
                    else:
                        ok_comb_neg = False 
                elif val in sensitive_dict[att]['P']['values_list']:
                    if sensitive_dict[att]['P'][val]['N_curr'] < sensitive_dict[att]['P'][val]['N_exp']:
                        constraint_neg.append((get_index(att, attributes), val))
                    else:
                        ok_comb_neg = False
                        
                if val in sensitive_dict[att]['D']['values_list']:
                    if sensitive_dict[att]['D'][val]['P_curr'] < sensitive_dict[att]['D'][val]['P_exp']:
                        constraint_pos.append((get_index(att, attributes), val))
                    else:
                        ok_comb_pos = False                        
                elif val in sensitive_dict[att]['P']['values_list']:
                    if sensitive_dict[att]['P'][val]['P_curr'] < sensitive_dict[att]['P'][val]['P_exp']:
                        constraint_pos.append((get_index(att, attributes), val))
                    else:
                        ok_comb_pos = False   
            
            if ok_comb_neg == True:
                constraints.append(tuple(constraint_neg))
                for tup in constraint_neg[1:]:
                    att = attributes[tup[0]]
                    val = tup[1]
                    if val in sensitive_dict[att]['D']['values_list']:
                        sensitive_dict[att]['D'][val]['N_curr'] = sensitive_dict[att]['D'][val]['N_curr'] + 1
                    elif val in sensitive_dict[att]['P']['values_list']:
                        sensitive_dict[att]['P'][val]['N_curr'] = sensitive_dict[att]['P'][val]['N_curr'] + 1

            if ok_comb_pos == True:
                constraints.append(tuple(constraint_pos))
                for tup in constraint_pos[1:]:
                    att = attributes[tup[0]]
                    val = tup[1]
                    if val in sensitive_dict[att]['D']['values_list']:
                        sensitive_dict[att]['D'][val]['P_curr'] = sensitive_dict[att]['D'][val]['P_curr'] + 1
                    elif val in sensitive_dict[att]['P']['values_list']:
                        sensitive_dict[att]['P'][val]['P_curr'] = sensitive_dict[att]['P'][val]['P_curr'] + 1
            
            if ok_comb_neg == False and ok_comb_pos == False:
                break
                
            if len(constraints) == starting_const_len:
                break

    #print("XXX Printing stuff for debugging purposes...")
    #for att in sensitive_attributes:
     #   for val in sensitive_dict[att]['D']['values_list']:
      #      DN_curr = sensitive_dict[att]['D'][val]['N_curr']
       #     DN_exp = sensitive_dict[att]['D'][val]['N_exp']
        #    DP_curr = sensitive_dict[att]['D'][val]['P_curr']
         #   DP_exp = sensitive_dict[att]['D'][val]['P_exp']            
          #  print(att, val, "DN_cur", DN_curr, "DN_exp", DN_exp, "DP_cur", DP_curr, "DP_exp", DP_exp)
        #for val in sensitive_dict[att]['P']['values_list']: 
         #   PN_curr = sensitive_dict[att]['P'][val]['N_curr']
          #  PN_exp = sensitive_dict[att]['P'][val]['N_exp']
           # PP_curr = sensitive_dict[att]['P'][val]['P_curr']
            #PP_exp = sensitive_dict[att]['P'][val]['P_exp']            
            #print(att, val, "PN_cur", PN_curr, "PN_exp", PN_exp, "PP_cur", PP_curr, "PP_exp", PP_exp)         

    ### PART4 - Creating new records using the constraints to balance the dataset
    
    constraints = Counter(tuple(constraints))
            
    for const in constraints.keys():
        
        subset = og_df.copy() #subgroup of records with const values (e.g.: Black, Male, Positive)
        for tup in const:
            subset = subset[subset[subset.columns[tup[0]]] == tup[1]]
            
        if len(subset) > 0:
            kmedoids = KMedoids(n_clusters=1, random_state=42).fit(subset)
            medoid = kmedoids.cluster_centers_[0] #medoid of subgroup
        else:
            medoid = [None]
        

        #we use a GA for every const [eg. Black, Male, Positve] and each associated number [e.g. 5]
        #e.g., we create 5 records following the consts
        #if we have 20 unique consts, we use the GA 20 times
        
        new_records = GA(values, const, constraints[const], forest, medoid, 
                         values_in_dataset_indexes, discrete_indexes, regular_indexes, causal_reg, 
                         causal_class, mode, ds)

        for all_records in new_records:
            for record in all_records:
                target = const[0][1]
                for tup in const[1:]:
                    att = attributes[tup[0]]
                    val = tup[1]
                    if val in sensitive_dict[att]['D']['values_list']:
                        if target == 0: 
                            sensitive_dict[att]['D'][val]['N'].append(record)
                        else:
                            sensitive_dict[att]['D'][val]['P'].append(record)
                    elif val in sensitive_dict[att]['P']['values_list']:
                        if target == 0: 
                            sensitive_dict[att]['P'][val]['N'].append(record)
                        else:
                            sensitive_dict[att]['P'][val]['P'].append(record)
                    else:
                        print ("ERROR! ERROR!")
                        print ("Value", val, "shouldn't exist for attribute", att)
                        
                X_proba.append(record) #balanced dataset (OG + Syntethic)
                genetic_data.append(record) #other dataset with ONLY synthetic data
       
    print ("=== NEW DATASET ===")
    final_df = pd.DataFrame.from_records(X_proba)
    final_df.columns = attributes + [class_name]
    
    genetic_df = pd.DataFrame.from_records(genetic_data)
    genetic_df.columns = attributes + [class_name]
    
    
    get_discrimination (final_df, sensitive_attributes, class_name)

    print("")
    print("OG dataset length:", len(df))
    print("Records generated:", len(genetic_df))
    print("New dataset length:", len(final_df))
    print("Time:", (time.time() - start_time))
    
    return final_df, genetic_df

In [ ]:
german = pd.read_json("german_train.json")

german

In [ ]:
sensitive_attributes = ['Sex', 'Age']
class_name = 'Risk'

causal_reg = []
causal_class = [[['Job'], 'Housing']] 
#multiple independent features are supported, e.g. [[['Job', 'Credit amount'], 'Housing']]

discrete_attributes = []
values_in_dataset_attributes = ['Job', 'Housing', 'Saving accounts', 'Checking account', 'Credit amount', 'Duration', 'Purpose']

In [ ]:
df_fair, df_genetic = main(german, sensitive_attributes, class_name, #<- must be included
                           causal_reg, causal_class, #<- causal relations
                           discrete_attributes, values_in_dataset_attributes, #<- how to handle the various features
                          'Outlier', 'Fixed') #<- fitness mode, datatased values mode